PURPOSE OF CODE

The test set scores are assigned and error is calculated as new columns for all test phrases.

PROCESS NOTES

WILL BE DONE
- recursive search for word groups may be written
- most dominant words may extracted for better score assginment

DONE
- test phrase is searched in train set
- if no found, in test phrase word group, try to find longest sub phrase that is in train set
- if sub phrase found in train set, search for remaining words seperately on train set, if no found get the closest word's sentiment score using w2v model created for each one and average all scores get.

In [1]:
import numpy as np
import gensim, logging
import pandas as pd
model= gensim.models.Word2Vec.load("w2vmodel")
words=pd.read_csv('data/words.csv',sep="\t")
train=pd.read_csv('data/train-clean.csv',sep="\t")
test=pd.read_csv('data/test.csv',sep="\t")

C:\Users\efear\Anaconda3\lib\site-packages\gensim-2.1.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


In [ ]:
test['calculation'] = pd.Series(np.NaN , index=test.index)
test['error'] = pd.Series(np.NaN , index=test.index)
test.head()

,Unnamed: 0,Unnamed: 0.1,PhraseId,SentenceId,Phrase,Sentiment,calculation,error
0,0,0,39654,1893,someth new,3,NaN,NaN
1,1,1,56036,2802,film editor,2,NaN,NaN
2,2,2,140518,7625,unusu biopic document,3,NaN,NaN
3,3,3,129586,6968,hi brawni frame cool compos deliveri fit bill ...,2,NaN,NaN
4,4,4,104116,5490,skip thi dreck rent anim hous go back sourc,1,NaN,NaN


In [ ]:
from IPython.core.debugger import Tracer; debug = Tracer()
from scipy.spatial import distance
  

phrase=[]
tmp1_phrase=[]
tmp2_phrase=[]
attribution_count=0
attribution_sum=0


for indx,row in test.iterrows():
    #check if phrase is in string type
    if isinstance(row.Phrase,str):
        #check if there is something
        if len(row.Phrase):
            #chech if that phrase is in the train set alreadt
            find=train.Phrase[train.Phrase==row.Phrase]
            if len(find.index):
                #if already in train set assign the score directly
                test.loc[indx,"calculation"]=train.loc[find.index[0],"Sentiment"]
            else:
                attribution_count=0
                attribution_sum=0
                phrase=row.Phrase.split()
                length=len(phrase)
                #if phrase contains single word
                if length>1:
                    tmp=1
                    # with tmp+1<length words could be checked seperately
                    while tmp<length:
                        for x in range(0,tmp+1):
                            #devide phrase into 2 part one is the major word group that will be searched as it is
                            #other group words will be scored seperately
                            tmp1_phrase=phrase[x:(length-tmp+x)]
                            tmp2_phrase=list(set(phrase)-set(tmp1_phrase))
                            find=train.Phrase[train.Phrase==" ".join(tmp1_phrase)]
                            a=len(find.index)
                            #check if word group is found if found break
                            if a:
                                break
                        #if found calculate score
                        if a:
                            attribution_count+=1
                            attribution_sum+=train.loc[find.index[0],"Sentiment"]
                            for y in tmp2_phrase:
                                find=train.Phrase[train.Phrase==y]
                                if len(find.index):
                                    attribution_count+=1
                                    attribution_sum+=train.loc[find.index[0],"Sentiment"]
                                else:
                                    try:
                                        vec=model[y]
                                        tmpvec=model[words.loc[0,"Phrase"]]
                                        tmpindx=0
                                        dist=distance.euclidean(vec,tmpvec)
                                        for indx1,row in words.iterrows():
                                            tmpdist=distance.euclidean(vec,model[row.Phrase])
                                            if tmpdist<dist:
                                                dist=tmpdist
                                                tmpvec=model[row.Phrase]
                                                tmpindx=indx1
                                        attribution_count+=1
                                        attribution_sum+=words.loc[tmpindx,"Sentiment"]
                                    except:
                                        pass
                            break
                        tmp+=1
                #if single word phrase calculate score
                else:
                    try:
                        vec=model[row.Phrase]
                        tmpvec=model[words.loc[0,"Phrase"]]
                        tmpindx=0
                        dist=distance.euclidean(vec,tmpvec)
                        for indx1,row in words.iterrows():
                            tmpdist=distance.euclidean(vec,model[row.Phrase])
                            if tmpdist<dist:
                                dist=tmpdist
                                tmpvec=model[row.Phrase]
                                tmpindx=indx1
                        attribution_count+=1
                        attribution_sum+=words.loc[tmpindx,"Sentiment"]
                    except:
                        pass
                if attribution_count!=0:
                    test.loc[indx,"calculation"]=attribution_sum/attribution_count

In [ ]:
test.head()

In [ ]:
for indx,row in test.iterrows():
    test.loc[indx,"error"]= abs(row.Sentiment-row.calculation)
test.head()

In [ ]:
test = test.drop('Unnamed: 0', 1)
test.to_csv("data/test-calculated.csv", sep='\t')